# Challenge Data Science

Desafio [Alura](https://www.alura.com.br/) -  Um projeto de Data Science com PySpark


##  Semana 01

## Transformando dados com PySpark
Vamos explorar e tratar uma base de dados que veio dos sistemas internos da nossa empresa.

### Imports e carregamento dos Dados

In [23]:
from pyspark.sql import(
    SparkSession, 
    functions as F,
    DataFrame
) 
import utils

In [24]:
spark = (
    SparkSession
    .builder
    .appName('InsightPlaces')
    .getOrCreate()
)

spark

In [25]:
data = (
    spark.read.format('json')
    .option('inferSchema', True)
    .load('data/semana_01/dataset_bruto.json')
)


### Exploração dos Dados

In [26]:
data.show(5)

+--------------------+--------------------+--------------------+
|             anuncio|             imagens|             usuario|
+--------------------+--------------------+--------------------+
|{0, [], [16], [0]...|[{39d6282a-71f3-4...|{9d44563d-3405-4e...|
|{0, [], [14], [0]...|[{23d2b3ab-45b0-4...|{36245be7-70fe-40...|
|{0, [1026], [1026...|[{1da65baa-368b-4...|{9dc415d8-1397-4d...|
|{0, [120], [120],...|[{79b542c6-49b4-4...|{9911a2df-f299-4a...|
|{0, [3], [3], [0]...|[{e2bc497b-6510-4...|{240a7aab-12e5-40...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [27]:

print('Número de linhas e colunas:', (data.count(), len(data.columns)) )
utils.printSeparator()
data.printSchema()

Número de linhas e colunas: (89083, 3)
root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-

### Análise e Tratamento dos Dados

In [28]:

data.select('anuncio.*', 'imagens', 'usuario.*').show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+--------------------+--------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|             imagens|                  id|    nome|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+--------------------+--------+
|    0|        []|     [16]|      [0]|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|[{260, 107, Venda...|[{39d6282a-71f3-4...|9d44563d-3405-4e8...|   Frank|
|    0|        []|     [14]|      [0]|                  []|{Centro, 2005

In [29]:
# +-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+--------------------+--------+
# |andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|             imagens|                  id|    nome|
# +-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+--------------------+--------+
# |    0|        []|     [16]|      [0]|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|[{260, 107, Venda...|[{39d6282a-71f3-4...|9d44563d-3405-4e8...|   Frank|
# |    0|        []|     [14]|      [0]|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|[{260, 107, Venda...|[{23d2b3ab-45b0-4...|36245be7-70fe-40c...|Caroline|
# |    0|    [1026]|   [1026]|      [0]|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4a1...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|[{null, 1613, Ven...|[{1da65baa-368b-4...|9dc415d8-1397-4d8...|  Oliver|
# |    0|     [120]|    [120]|      [0]|[Portão eletrônic...|{Campo Grande, 23...|ac6cb348-69d6-45a...|    [0]|    []|       Usado|      Outros|Residencial|  []|[{80, null, Venda...|[{79b542c6-49b4-4...|9911a2df-f299-4a7...| Matthew|
# |    0|       [3]|      [3]|      [0]|                  []|{São Cristóvão, 2...|e032b908-ef42-4d4...|    [0]|    []|       Usado|      Outros|Residencial|  []|[{0, 0, Venda, 50...|[{e2bc497b-6510-4...|240a7aab-12e5-406...|    Jude|
# +-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+--------------------+--------+

In [30]:
# Explode na coluna anuncio e salvando em novo df

data_in_prepared = data.select(
    'anuncio.andar',
    'anuncio.area_total',
    'anuncio.area_util',
    'anuncio.banheiros',
    'anuncio.caracteristicas',
    'anuncio.endereco.*',
    F.col('anuncio.id').alias('id_anuncio'),
    'anuncio.quartos',
    'anuncio.suites',
    'anuncio.tipo_anuncio',
    'anuncio.tipo_unidade',
    'anuncio.tipo_uso',
    'anuncio.vaga',
    'anuncio.valores.condominio',
    'anuncio.valores.iptu',
    'anuncio.valores.tipo',
    'anuncio.valores.valor'
 )
 
data_in_prepared.show(3)

# +-----+----------+---------+---------+---------------+--------------+--------+--------------+--------------+----------+----------+----+-----------------+------------+--------------------+-------+------+------------+------------+---------+----+----------+------+-------+-------+
# |andar|area_total|area_util|banheiros|caracteristicas|        bairro|     cep|        cidade|        estado|  latitude| longitude|pais|              rua|        zona|          id_anuncio|quartos|suites|tipo_anuncio|tipo_unidade| tipo_uso|vaga|condominio|  iptu|   tipo|  valor|
# +-----+----------+---------+---------+---------------+--------------+--------+--------------+--------------+----------+----------+----+-----------------+------------+--------------------+-------+------+------------+------------+---------+----+----------+------+-------+-------+
# |    0|        []|     [16]|      [0]|             []|        Centro|20061003|Rio de Janeiro|Rio de Janeiro|-22.906082| -43.18671|  BR| Rua Buenos Aires|Zona Central|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|Comercial| [1]|     [260]| [107]|[Venda]|[10000]|
# |    0|        []|     [14]|      [0]|             []|        Centro|20051040|Rio de Janeiro|Rio de Janeiro|-22.902536| -43.18434|  BR|   Avenida Passos|Zona Central|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|Comercial| [0]|     [260]| [107]|[Venda]|[10000]|
# |    0|    [1026]|   [1026]|      [0]|             []|Maria da Graça|20785360|Rio de Janeiro|Rio de Janeiro|-22.885516|-43.269083|  BR|Rua Luís de Brito|  Zona Norte|1fb030a5-9e3e-4a1...|    [0]|    []|       Usado|      Outros|Comercial| [0]|    [null]|[1613]|[Venda]|[10000]|
# +-----+----------+---------+---------+---------------+--------------+--------+--------------+--------------+----------+----------+----+-----------------+------------+--------------------+-------+------+------------+------------+---------+----+----------+------+-------+-------+

+-----+----------+---------+---------+---------------+--------------+--------+--------------+--------------+----------+----------+----+-----------------+------------+--------------------+-------+------+------------+------------+---------+----+----------+------+-------+-------+
|andar|area_total|area_util|banheiros|caracteristicas|        bairro|     cep|        cidade|        estado|  latitude| longitude|pais|              rua|        zona|          id_anuncio|quartos|suites|tipo_anuncio|tipo_unidade| tipo_uso|vaga|condominio|  iptu|   tipo|  valor|
+-----+----------+---------+---------+---------------+--------------+--------+--------------+--------------+----------+----------+----+-----------------+------------+--------------------+-------+------+------------+------------+---------+----+----------+------+-------+-------+
|    0|        []|     [16]|      [0]|             []|        Centro|20061003|Rio de Janeiro|Rio de Janeiro|-22.906082| -43.18671|  BR| Rua Buenos Aires|Zona Central|

O time de Data Science solicitou que fizéssemos alguns **filtros** nas colunas tipo_uso, tipo_unidade e tipo_anuncio da nossa base de dados:

- tipo_uso: Residencial;
- tipo_unidade: Apartamento;
- tipo_anuncio: Usado.

Verificando as frequências das colunas das colunas selecionadas antes de filtrá-las.

In [31]:
colunas_frequencia = ['tipo_uso', 'tipo_unidade', 'tipo_anuncio']

In [32]:
data_select_tmp = (
    data_in_prepared.select(colunas_frequencia)
)

In [33]:
for c in colunas_frequencia:
    print(data_in_prepared.groupBy(c).count().show())


+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4542|
|Residencial|84541|
+-----------+-----+

None


+------------+-----+
|tipo_unidade|count|
+------------+-----+
|      Outros|11963|
| Apartamento|66801|
|        Casa|10319|
+------------+-----+

None


+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|88827|
|  Lançamento|  256|
+------------+-----+

None


In [34]:
# Efetuando o Filtro

data_in_prepared = (
    data_in_prepared.where(
        (data_in_prepared.tipo_uso == 'Residencial') 
        & (data_in_prepared.tipo_unidade == 'Apartamento')
        & (data_in_prepared.tipo_anuncio == 'Usado')
    )
)

In [35]:
# Convertendo os tipos de dados

data_in_prepared = (
    data_in_prepared
    .withColumn('area_total', F.concat_ws(',', F.col('area_total')).cast('double') )
    .withColumn('area_util', F.concat_ws(',', F.col('area_util')).cast('double') )
    .withColumn('banheiros', F.concat_ws(',', F.col('banheiros')).cast('double') )
    .withColumn('caracteristicas', F.concat_ws(' - ', F.col('caracteristicas')) )
    .withColumn('quartos', F.concat_ws(',', F.col('quartos')).cast('double') )
    .withColumn('suites', F.concat_ws(',', F.col('suites')).cast('double') )
    .withColumn('vaga', F.concat_ws(',', F.col('vaga')).cast('double') )
    .withColumn('condominio', F.concat_ws(',', F.col('condominio')).cast('double') )
    .withColumn('iptu', F.concat_ws(',', F.col('iptu')).cast('double') )
    .withColumn('tipo', F.concat_ws(',', F.col('tipo')) )
    .withColumn('valor', F.concat_ws(',', F.col('valor')).cast('double') )

)

data_in_prepared.show(5)

# +-----+----------+---------+---------+--------------------+---------+--------+--------------+--------------+----------+----------+----+--------------------+----------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-------+
# |andar|area_total|area_util|banheiros|     caracteristicas|   bairro|     cep|        cidade|        estado|  latitude| longitude|pais|                 rua|      zona|          id_anuncio|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|condominio|iptu| tipo|  valor|
# +-----+----------+---------+---------+--------------------+---------+--------+--------------+--------------+----------+----------+----+--------------------+----------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-------+
# |    3|      43.0|     43.0|      1.0|Academia - Churra...|Paciência|23585430|Rio de Janeiro|Rio de Janeiro|-22.919851|-43.634034|  BR|Estrada de Santa ...|Zona Oeste|d2e3a3aa-09b5-45a...|    2.0|  null|       Usado| Apartamento|Residencial| 1.0|     245.0|null|Venda|15000.0|
# |    2|      42.0|     42.0|      1.0|Churrasqueira - P...|Paciência|23585430|Rio de Janeiro|Rio de Janeiro|-22.928108|-43.635375|  BR|Estrada de Santa ...|Zona Oeste|085bab2c-87ad-452...|    2.0|  null|       Usado| Apartamento|Residencial| 1.0|       0.0| 0.0|Venda|15000.0|
# |    1|      41.0|     41.0|      1.0|Portaria 24h - Co...|Guaratiba|23036060|Rio de Janeiro|Rio de Janeiro|-22.948756|-43.582824|  BR|Estrada Cabuçu de...|Zona Oeste|18d22cbe-1b86-476...|    2.0|  null|       Usado| Apartamento|Residencial| 1.0|       0.0| 0.0|Venda|20000.0|
# |    3|      43.0|     43.0|      1.0|Churrasqueira - P...|   Cosmos|23066271|Rio de Janeiro|Rio de Janeiro|-22.888194|-43.629602|  BR|Estrada da Paciência|Zona Oeste|bed8a354-9317-442...|    2.0|  null|       Usado| Apartamento|Residencial| 0.0|     285.0|null|Venda|20000.0|
# |    2|      43.0|     43.0|      1.0|Academia - Churra...|Guaratiba|23036060|Rio de Janeiro|Rio de Janeiro|-22.948291|-43.582205|  BR|Estrada Cabuçu de...|Zona Oeste|12a13315-d67f-48f...|    2.0|  null|       Usado| Apartamento|Residencial| 1.0|     245.0|null|Venda|15000.0|
# +-----+----------+---------+---------+--------------------+---------+--------+--------------+--------------+----------+----------+----+--------------------+----------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-------+

+-----+----------+---------+---------+--------------------+---------+--------+--------------+--------------+----------+----------+----+--------------------+----------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-------+
|andar|area_total|area_util|banheiros|     caracteristicas|   bairro|     cep|        cidade|        estado|  latitude| longitude|pais|                 rua|      zona|          id_anuncio|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|condominio|iptu| tipo|  valor|
+-----+----------+---------+---------+--------------------+---------+--------+--------------+--------------+----------+----------+----+--------------------+----------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-------+
|    3|      43.0|     43.0|      1.0|Academia - Churra...|Paciência|23585430|Rio de Janeiro|Rio de Janeiro|-22.919851|-43.634034|  BR|Estrada de Santa ...|Zona Oeste|d2

#### Tratamento de informações sobre localização

In [36]:
'''A equipe de ciência de dados nos solicitou que apenas as informações 
sobre bairro e zona da cidade fossem extraídas.

Então, vamos analisar a coluna endereço e transformar apenas as 
informações sobre bairro e zona em colunas de nosso DataFrame.
'''

data_in_prepared = (
    data_in_prepared
    .drop('cep')
    .drop('cidade')
    .drop('estado')
    .drop('latitude')
    .drop('longitude')
    .drop('pais')
    .drop('rua')
)

In [37]:
data_in_prepared.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: double (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: double (nullable = true)
 |-- caracteristicas: string (nullable = false)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- id_anuncio: string (nullable = true)
 |-- quartos: double (nullable = true)
 |-- suites: double (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: double (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- tipo: string (nullable = false)
 |-- valor: double (nullable = true)



In [38]:
data_in_prepared.show(10)

+-----+----------+---------+---------+--------------------+---------+----------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-------+
|andar|area_total|area_util|banheiros|     caracteristicas|   bairro|      zona|          id_anuncio|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|condominio|iptu| tipo|  valor|
+-----+----------+---------+---------+--------------------+---------+----------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-------+
|    3|      43.0|     43.0|      1.0|Academia - Churra...|Paciência|Zona Oeste|d2e3a3aa-09b5-45a...|    2.0|  null|       Usado| Apartamento|Residencial| 1.0|     245.0|null|Venda|15000.0|
|    2|      42.0|     42.0|      1.0|Churrasqueira - P...|Paciência|Zona Oeste|085bab2c-87ad-452...|    2.0|  null|       Usado| Apartamento|Residencial| 1.0|       0.0| 0.0|Venda|15000.0|
|    1|      41.0|     41.0|      1.0|Portaria 24h

#### Salvando os dados no formato parquet

In [39]:
data_in_prepared.write.mode('overwrite').parquet('data/semana_02/dataset_tratado')